<a href="https://colab.research.google.com/github/dolmani38/Summary2/blob/main/sentence_complete_0831.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 불완전 문장(일부 토큰이 빠진)을 완성형 문장으로 교정하는 Transformer

In [1]:
if True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install transformers
!pip3 install sentence-transformers

     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 636 kB 56.4 MB/s 
     |████████████████████████████████| 3.3 MB 39.5 MB/s 
     |████████████████████████████████| 895 kB 41.6 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 1.2 MB 37.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=48a8d46cf7cb9787c48c382864aa82076211a1bc9e67d81955252de6a7c99a8b
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers


In [3]:
import matplotlib.pyplot as plt
# set seeds for reproducability
from numpy.random import seed
#seed(1)

import pandas as pd
import numpy as np
import string, os 

import urllib.request
import nltk
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [5]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

## data 준비

In [ ]:
import json  
import zipfile  

data_1 = None  
data = None  
with zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/summary/data/사설잡지_2.train_original.json.zip", "r") as z:
    with z.open('train_original.json') as f:  
        data = f.read()  
        data_1 = json.loads(data.decode("utf-8"))

In [ ]:
data_2 = None  
data = None  
with zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/summary/data/신문기사_2.train_original.json.zip", "r") as z:
    with z.open('train_original.json') as f:  
        data = f.read()  
        data_2 = json.loads(data.decode("utf-8"))

In [ ]:
' '.join(data_2[1]['article_original'])

'서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 우수의정대상을 받았다. 가충순 의원과 이수의 의원은 16일 팔봉면 폰타나 리조트에서 열린 한국지역신문협회 하계 워크샵에서 지역사회 발전을 위해 활발한 의정활동을 펼친 공로를 인정받아 우수의정대상을 수상했다. 지난해 6월 제7회 전국동시지방선거를 통해 등원한 두 의원은 산업건설위원회에서 열정적인 의정활동을 펼치고 있다. 가충순 의원은 5분발언, 행정사무감사, 시정질문을 통해 자동차 연비테스트 연구시설 유치, 천수만 염해피해 재발 방지, 서산시 대표 농산물 육성 등 지역의 크고 작은 문제를 개선하기 위해 노력하고 있다. 이수의 의원은 지난 행정사무감사에서 대산공단 기업 임원을 참고인으로 출석시켜 지역인재채용 및 관내업체·자재 활용을 확대할 것을 제안하며 기존 행정사무감사의 틀을 깨는 등 다양한 의정활동을 펼쳐나가고 있다. 가충순 의원은 "시의원이라면 마땅히 해야할 일을 한 것 뿐인데 상까지 주시니 몸 둘 바를 모르겠다"며 "항상 초심을 잊지 않고 지역 발전을 위해 최선을 다하겠다"고 소감을 밝혔다. 이수의 의원은 "믿고 뽑아주신 주민들을 위해 당연히 해야할 일을 했을 뿐인데 상까지 받게 돼 영광"이라며 "시민들이 자부심을 느낄 수 있는 지역사회를 만들어 나가기 위해 앞으로도 최선을 다 하겠다"고 말했다.'

In [ ]:
sentences = []
for i in range(len(data_2)):
    for txt in data_2[i]['article_original']:
        sentences.append(txt)
for i in range(len(data_1)):
    for txt in data_1[i]['article_original']:
        sentences.append(txt)        

In [ ]:
len(sentences)

4407160

## 문법 Scoring 모델

In [ ]:
from transformers import BertTokenizer, BertTokenizerFast,AutoTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split

import time
import random
import datetime

# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    #txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    txt = txt.replace(' .','.')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()

def shuffling(txt):
    txt_list = txt.split(' ')
    random.shuffle(txt_list)
    return ' '.join(txt_list)

def collect_training_dataset_for_grammar_discriminator(sentences_dataset):

    sentences = []
    labels = []

    for txtss in sentences_dataset:
        txtss = clean_text(txtss)
        txts = txtss.strip().split('.')
        for txt in txts:  
            txt = txt.strip()
            if len(txt) > 10:
                #ko_grammar_dataset.append([txt,1])
                txt = txt.replace('.','')
                tf = random.choice([True,False])
                # 정상 또는 비정상 둘중에 하나만 데이터셋에 추가
                if (tf):
                    sentences.append(txt) # '.'의 위치를 보고 True, False를 판단 하기 땜에...
                    labels.append(1)
                else:
                    sentences.append(shuffling(txt))
                    labels.append(0)

    return sentences,labels

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

class Grammar_Discriminator:


    def __init__(self, pretraoned_kobert_model_name='kykim/bert-kor-base', input_dir=None):

        if input_dir is None:
            self.tokenizer = BertTokenizerFast.from_pretrained(pretraoned_kobert_model_name)
            self.discriminator = BertForSequenceClassification.from_pretrained(
                                    pretraoned_kobert_model_name, # Use the 12-layer BERT model, with an uncased vocab.
                                    num_labels = 2, # The number of output labels--2 for binary classification.
                                                    # You can increase this for multi-class tasks.   
                                    output_attentions = False, # Whether the model returns attentions weights.
                                    output_hidden_states = False, # Whether the model returns all hidden-states.
                                )            
        else:
            self.__load_model(input_dir)




    def set_dataset(self, sentences,labels):
        # Print the original sentence.
        print(' Original: ', sentences[0])

        # Tokenize all of the sentences and map the tokens to thier word IDs.
        input_ids = []
        attention_masks = []

        # For every sentence...
        for sent in sentences:
            # `encode_plus` will:
            #   (1) Tokenize the sentence.
            #   (2) Prepend the `[CLS]` token to the start.
            #   (3) Append the `[SEP]` token to the end.
            #   (4) Map tokens to their IDs.
            #   (5) Pad or truncate the sentence to `max_length`
            #   (6) Create attention masks for [PAD] tokens.
            encoded_dict = self.tokenizer.encode_plus(
                                sent,                      # Sentence to encode.
                                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                max_length = 64,           # Pad & truncate all sentences.
                                pad_to_max_length = True,
                                return_attention_mask = True,   # Construct attn. masks.
                                return_tensors = 'pt',     # Return pytorch tensors.
                                truncation = True,
                        )
            
            # Add the encoded sentence to the list.    
            input_ids.append(encoded_dict['input_ids'])
            
            # And its attention mask (simply differentiates padding from non-padding).
            attention_masks.append(encoded_dict['attention_mask'])

        # Convert the lists into tensors.
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)
        labels = torch.tensor(labels)

        # Print sentence 0, now as a list of IDs.
        print('Original: ', sentences[0])
        print('Token IDs:', input_ids[0])

        # Training & Validation Split
        # Divide up our training set to use 90% for training and 10% for validation.

        # Combine the training inputs into a TensorDataset.
        dataset = TensorDataset(input_ids, attention_masks, labels)

        # Create a 90-10 train-validation split.

        # Calculate the number of samples to include in each set.
        train_size = int(0.9 * len(dataset))
        val_size = len(dataset) - train_size

        # Divide the dataset by randomly selecting samples.
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        print('{:>5,} training samples'.format(train_size))
        print('{:>5,} validation samples'.format(val_size))

        # The DataLoader needs to know our batch size for training, so we specify it 
        # here. For fine-tuning BERT on a specific task, the authors recommend a batch 
        # size of 16 or 32.
        self.batch_size = 32

        # Create the DataLoaders for our training and validation sets.
        # We'll take training samples in random order. 
        self.train_dataloader = DataLoader(
                    train_dataset,  # The training samples.
                    sampler = RandomSampler(train_dataset), # Select batches randomly
                    batch_size = self.batch_size # Trains with this batch size.
                )

        # For validation the order doesn't matter, so we'll just read them sequentially.
        self.validation_dataloader = DataLoader(
                    val_dataset, # The validation samples.
                    sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
                    batch_size = self.batch_size # Evaluate with this batch size.
                )        



    def train(self,epochs=4):
        # Tell pytorch to run this model on the GPU.
        self.discriminator.cuda()

        # Get all of the model's parameters as a list of tuples.
        params = list(self.discriminator.named_parameters())

        print('The BERT model has {:} different named parameters.\n'.format(len(params)))

        print('==== Embedding Layer ====\n')

        for p in params[0:5]:
            print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

        print('\n==== First Transformer ====\n')

        for p in params[5:21]:
            print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

        print('\n==== Output Layer ====\n')

        for p in params[-4:]:
            print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))  

        # Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
        # I believe the 'W' stands for 'Weight Decay fix"
        self.optimizer = AdamW(self.discriminator.parameters(),
                        lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                        eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                        )

        # Number of training epochs. The BERT authors recommend between 2 and 4. 
        # We chose to run for 4, but we'll see later that this may be over-fitting the
        # training data.
        #epochs = 2

        # Total number of training steps is [number of batches] x [number of epochs]. 
        # (Note that this is not the same as the number of training samples).
        total_steps = len(self.train_dataloader) * epochs

        # Create the learning rate scheduler.
        scheduler = get_linear_schedule_with_warmup(self.optimizer, 
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = total_steps)
            
        # This training code is based on the `run_glue.py` script here:
        # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

        # Set the seed value all over the place to make this reproducible.
        seed_val = 42

        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)

        # We'll store a number of quantities such as training and validation loss, 
        # validation accuracy, and timings.
        training_stats = []

        # Measure the total training time for the whole run.
        total_t0 = time.time()

        # For each epoch...
        for epoch_i in range(0, epochs):
            
            # ========================================
            #               Training
            # ========================================
            
            # Perform one full pass over the training set.

            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            print('Training...')

            # Measure how long the training epoch takes.
            t0 = time.time()

            # Reset the total loss for this epoch.
            total_train_loss = 0

            # Put the model into training mode. Don't be mislead--the call to 
            # `train` just changes the *mode*, it doesn't *perform* the training.
            # `dropout` and `batchnorm` layers behave differently during training
            # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
            self.discriminator.train()

            # For each batch of training data...
            for step, batch in enumerate(self.train_dataloader):

                # Progress update every 40 batches.
                if step % 40 == 0 and not step == 0:
                    # Calculate elapsed time in minutes.
                    elapsed = format_time(time.time() - t0)
                    
                    # Report progress.
                    print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(self.train_dataloader), elapsed))

                # Unpack this training batch from our dataloader. 
                #
                # As we unpack the batch, we'll also copy each tensor to the GPU using the 
                # `to` method.
                #
                # `batch` contains three pytorch tensors:
                #   [0]: input ids 
                #   [1]: attention masks
                #   [2]: labels 
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)

                # Always clear any previously calculated gradients before performing a
                # backward pass. PyTorch doesn't do this automatically because 
                # accumulating the gradients is "convenient while training RNNs". 
                # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
                self.discriminator.zero_grad()        

                # Perform a forward pass (evaluate the model on this training batch).
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # It returns different numbers of parameters depending on what arguments
                # arge given and what flags are set. For our useage here, it returns
                # the loss (because we provided labels) and the "logits"--the model
                # outputs prior to activation.
                outputs = self.discriminator(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask, 
                                    labels=b_labels)
                loss, logits = outputs.loss, outputs.logits
                # Accumulate the training loss over all of the batches so that we can
                # calculate the average loss at the end. `loss` is a Tensor containing a
                # single value; the `.item()` function just returns the Python value 
                # from the tensor.
                total_train_loss += loss.item()

                # Perform a backward pass to calculate the gradients.
                loss.backward()

                # Clip the norm of the gradients to 1.0.
                # This is to help prevent the "exploding gradients" problem.
                torch.nn.utils.clip_grad_norm_(self.discriminator.parameters(), 1.0)

                # Update parameters and take a step using the computed gradient.
                # The optimizer dictates the "update rule"--how the parameters are
                # modified based on their gradients, the learning rate, etc.
                self.optimizer.step()

                # Update the learning rate.
                scheduler.step()

            # Calculate the average loss over all of the batches.
            avg_train_loss = total_train_loss / len(self.train_dataloader)            
            
            # Measure how long this epoch took.
            training_time = format_time(time.time() - t0)

            print("")
            print("  Average training loss: {0:.2f}".format(avg_train_loss))
            print("  Training epcoh took: {:}".format(training_time))
                
            # ========================================
            #               Validation
            # ========================================
            # After the completion of each training epoch, measure our performance on
            # our validation set.

            print("")
            print("Running Validation...")

            t0 = time.time()

            # Put the model in evaluation mode--the dropout layers behave differently
            # during evaluation.
            self.discriminator.eval()

            # Tracking variables 
            total_eval_accuracy = 0
            total_eval_loss = 0
            nb_eval_steps = 0

            # Evaluate data for one epoch
            for batch in self.validation_dataloader:
                
                # Unpack this training batch from our dataloader. 
                #
                # As we unpack the batch, we'll also copy each tensor to the GPU using 
                # the `to` method.
                #
                # `batch` contains three pytorch tensors:
                #   [0]: input ids 
                #   [1]: attention masks
                #   [2]: labels 
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)
                
                # Tell pytorch not to bother with constructing the compute graph during
                # the forward pass, since this is only needed for backprop (training).
                with torch.no_grad():        

                    # Forward pass, calculate logit predictions.
                    # token_type_ids is the same as the "segment ids", which 
                    # differentiates sentence 1 and 2 in 2-sentence tasks.
                    # The documentation for this `model` function is here: 
                    # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                    # Get the "logits" output by the model. The "logits" are the output
                    # values prior to applying an activation function like the softmax.
                    outputs = self.discriminator(b_input_ids, 
                                        token_type_ids=None, 
                                        attention_mask=b_input_mask,
                                        labels=b_labels)
                loss, logits = outputs.loss, outputs.logits
                # Accumulate the validation loss.
                total_eval_loss += loss.item()

                # Move logits and labels to CPU
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()

                # Calculate the accuracy for this batch of test sentences, and
                # accumulate it over all batches.
                total_eval_accuracy += flat_accuracy(logits, label_ids)
                

            # Report the final accuracy for this validation run.
            avg_val_accuracy = total_eval_accuracy / len(self.validation_dataloader)
            print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

            # Calculate the average loss over all of the batches.
            avg_val_loss = total_eval_loss / len(self.validation_dataloader)
            
            # Measure how long the validation run took.
            validation_time = format_time(time.time() - t0)
            
            print("  Validation Loss: {0:.2f}".format(avg_val_loss))
            print("  Validation took: {:}".format(validation_time))

            # Record all statistics from this epoch.
            training_stats.append(
                {
                    'epoch': epoch_i + 1,
                    'Training Loss': avg_train_loss,
                    'Valid. Loss': avg_val_loss,
                    'Valid. Accur.': avg_val_accuracy,
                    'Training Time': training_time,
                    'Validation Time': validation_time
                }
            )

        print("")
        print("Training complete!")

        print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
            

        return training_stats

    def save_model(self, output_dir = './model_save/'):
        # Create output directory if needed
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        print("Saving model to %s" % output_dir)

        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = self.discriminator.module if hasattr(self.discriminator, 'module') else self.discriminator  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        self.tokenizer.save_pretrained(output_dir)

        # Good practice: save your training arguments together with the trained model
        # torch.save(args, os.path.join(output_dir, 'training_args.bin'))

    def __load_model(self, input_dir = './drive/MyDrive/Colab Notebooks/summary/en_grammar_check_model'):
        print('Loading BERT tokenizer...')
        self.tokenizer = BertTokenizerFast.from_pretrained(input_dir)
        self.discriminator = BertForSequenceClassification.from_pretrained(input_dir)

    def transfer_learning(self, sentences, train_for = True):
        
        input_ids = []
        attention_masks = []

        # For every sentence...
        for sent in sentences:
            # `encode_plus` will:
            #   (1) Tokenize the sentence.
            #   (2) Prepend the `[CLS]` token to the start.
            #   (3) Append the `[SEP]` token to the end.
            #   (4) Map tokens to their IDs.
            #   (5) Pad or truncate the sentence to `max_length`
            #   (6) Create attention masks for [PAD] tokens.
            encoded_dict = self.tokenizer.encode_plus(
                                sent,                      # Sentence to encode.
                                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                max_length = 64,           # Pad & truncate all sentences.
                                pad_to_max_length = True,
                                return_attention_mask = True,   # Construct attn. masks.
                                return_tensors = 'pt',     # Return pytorch tensors.
                                truncation = True,
                        )
            # Add the encoded sentence to the list.    
            input_ids.append(encoded_dict['input_ids'])

            # And its attention mask (simply differentiates padding from non-padding).
            attention_masks.append(encoded_dict['attention_mask'])
        
        if train_for:
            b_labels = torch.ones(len(sentences),dtype=torch.long).to(device)
        else:
            b_labels = torch.zeros(len(sentences),dtype=torch.long).to(device)
        #print(b_labels)
        # Convert the lists into tensors.
        input_ids = torch.cat(input_ids, dim=0).to(device)
        attention_masks = torch.cat(attention_masks, dim=0).to(device)    
        #if str(discriminator1.device) == 'cpu':
        #    pass
        #else:
        #    input_ids = input_ids.to(device)
        #    attention_masks = attention_masks.to(device)        

        outputs = self.discriminator(input_ids, 
                            token_type_ids=None, 
                            attention_mask=attention_masks, 
                                labels=b_labels)

        #print(outputs)
        #return torch.sigmoid(outputs[0][:,1])
        #return outputs[0][:,1]
        return outputs['loss'], outputs['logits']


In [ ]:
grammer_checker = Grammar_Discriminator(input_dir = '/content/drive/MyDrive/Colab Notebooks/summary/ko_grammar_model3')

Loading BERT tokenizer...


KeyboardInterrupt: ignored

## dataset 만들기

In [ ]:
import random

def crush_sent(txt,crush_rate=0.3):
    sens = np.array(txt.split(' '))
    l = len(sens)
    r = int(l*crush_rate)
    s = [i for i in range(l)]
    c = random.sample(s,r)
    t = [x for x in s if (x not in c)]
    crushed_text = ' '.join(sens[t])
    score = 0 #grammer_checker.transfer_learning([crushed_text])[1][0,1].item()
    return crushed_text,score,txt

In [ ]:
crush_sent('가충순 의원과 이수의 의원은 16일 팔봉면 폰타나 리조트에서 열린 한국지역신문협회 하계 워크샵에서 지역사회 발전을 위해 활발한 의정활동을 펼친 공로를 인정받아 우수의정대상을 수상했다.')

('이수의 의원은 팔봉면 폰타나 열린 한국지역신문협회 워크샵에서 지역사회 발전을 활발한 의정활동을 펼친 공로를 인정받아 우수의정대상을 수상했다.',
 0,
 '가충순 의원과 이수의 의원은 16일 팔봉면 폰타나 리조트에서 열린 한국지역신문협회 하계 워크샵에서 지역사회 발전을 위해 활발한 의정활동을 펼친 공로를 인정받아 우수의정대상을 수상했다.')

In [ ]:
random.shuffle(sentences)

In [ ]:
import json
import pickle

dataset = []
for i,sentence in enumerate(sentences[:1000000]):
    print(f'\r {i+1}/{len(sentences)}', end="", flush=True)
    src,score,trg = crush_sent(sentence)
    dataset.append((src,score,trg))
    if i%5000 == 0:
        with open("/content/drive/MyDrive/GAN_ENDE/sentence_complete_dataset.bin", "wb") as fp:   #Pickling
            pickle.dump(dataset, fp)        

 2239/4407160

KeyboardInterrupt: ignored

In [ ]:
dataset[1000]

In [ ]:
import json
import pickle
if False:
    with open("/content/drive/MyDrive/GAN_ENDE/sentence_complete_dataset.bin", "wb") as fp:   #Pickling
        pickle.dump(dataset, fp)

In [ ]:
import json
import pickle
if False:
    with open("/content/drive/MyDrive/GAN_ENDE/sentence_complete_dataset.bin", "rb") as fp:   # Unpickling
        dataset = pickle.load(fp)

In [ ]:
len(dataset)

1

# 학습

In [6]:
pre_trained_kobert_model_name='kykim/bert-kor-base'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
from transformers import EncoderDecoderModel, BertTokenizerFast
import torch

tokenizer = BertTokenizerFast.from_pretrained(pre_trained_kobert_model_name)

Downloading:   0%|          | 0.00/344k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/725 [00:00<?, ?B/s]

In [8]:
op = tokenizer('옛날 어느 집에 귀여운 여자 아기가 태어났어요.[SEP]아기는 무럭무럭 자라서 예쁘고 마음씨 고운 소녀가 되었어요.', return_tensors="pt",padding="max_length", truncation=True, max_length=64)
print(op)
print("Tokens (str)      : {}".format([tokenizer.convert_ids_to_tokens(s) for s in op['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(op['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(op['attention_mask'].tolist()[0]))

{'input_ids': tensor([[    2, 17463, 14385, 14662, 15886, 14891, 17818, 33791, 13972,  2016,
             3, 35244,  4215,  8669,  8035,  8669, 19206,  8044, 17364, 14125,
          8472, 26268, 18857,  8048, 17292,  2016,     3,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
curren_sentence_num = 0
dataset_iterator = []

In [ ]:
import pickle

if False:
    with open("/content/drive/MyDrive/GAN_ENDE/sentence_complete_tokenized_dataset.bin", "wb") as fp:   #Pickling
        pickle.dump(dataset_iterator, fp)

In [ ]:

encoder_max_length = 64
decoder_max_length = 64
batch_size = 32 #4 # 64, 128

for cnt in range(curren_sentence_num,len(dataset),batch_size):
    print(f'\r {cnt+1}/{len(dataset)}', end="", flush=True)
    batch = {}
    batch['input_ids'] = []
    batch['attention_mask'] = []
    batch['decoder_input_ids'] = []
    batch['decoder_attention_mask'] = []
    batch['labels'] = []

    for i in range(cnt,cnt+batch_size):
        (src,score,trg) = dataset[i]
        inputs = tokenizer(src, padding="max_length", truncation=True, max_length=encoder_max_length)
        outputs = tokenizer(trg, padding="max_length", truncation=True, max_length=decoder_max_length)
        batch["input_ids"].append(inputs.input_ids)
        batch["attention_mask"].append(inputs.attention_mask)
        batch["decoder_input_ids"].append(outputs.input_ids)
        batch["decoder_attention_mask"].append(outputs.attention_mask)
        batch["labels"].append(outputs.input_ids.copy())
    
    dataset_iterator.append(batch)


In [10]:
len(dataset_iterator)

31250

In [ ]:
import pickle

if False:
    with open("/content/drive/MyDrive/GAN_ENDE/sentence_complete_tokenized_dataset.bin", "wb") as fp:   #Pickling
        pickle.dump(dataset_iterator, fp)

In [9]:
import pickle

if True:
    with open("/content/drive/MyDrive/GAN_ENDE/sentence_complete_tokenized_dataset.bin", "rb") as fp:   # Unpickling
        dataset_iterator = pickle.load(fp)

### 학습

In [11]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [12]:
def generate_summary(text):
    # cut off at BERT max length 512
    sens = nltk.sent_tokenize(text)
    assert(len(sens) == 2)
    inputs = tokenizer(sens[0].strip()+'[SEP]'+sens[1].strip(), padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    model.to(device)
    model.eval()
    outputs = model.generate(input_ids, attention_mask=attention_mask).cpu().detach().numpy()[0]
    o=[]
    for token in outputs:
        if token == tokenizer.pad_token_id:
            break
        o.append(token)
    output_str = tokenizer.batch_decode([o], skip_special_tokens=True)[0]

    return output_str

In [13]:
def generate_summary2(text):
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    model.to(device)
    model.eval()
    outputs = model.generate(input_ids, attention_mask=attention_mask).cpu().detach().numpy()[0]
    o=[]
    for token in outputs:
        if token == tokenizer.pad_token_id:
            break
        o.append(token)
    output_str = tokenizer.batch_decode([o], skip_special_tokens=True)[0]

    return output_str

In [14]:
from transformers import EncoderDecoderModel, BertTokenizerFast

try:
    del model
    print('delete model')
except Exception as ex:
    pass

use_pretrained_model = True

if use_pretrained_model:
    model = EncoderDecoderModel.from_pretrained("/content/drive/MyDrive/GAN_ENDE/sentence_complete_model")
    print('Load from pre-trained model...')
else:
    model = EncoderDecoderModel.from_encoder_decoder_pretrained(pre_trained_kobert_model_name, pre_trained_kobert_model_name) # initialize Bert2Bert from pre-trained checkpoints
    print('Initialize with ',pre_trained_kobert_model_name)

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

model.config.max_length = 142
model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

N_EPOCHS = 1

Load from pre-trained model...


In [19]:
print('\tTarget text:',generate_summary2('옛날 귀여운 여자 아기가 마음씨 소녀가 되었어요.')) 

	Target text: 옛날 귀여운 여자 아기가 마음씨 좋은 소녀가 되었어요. ( ) 라고 말했다고 합니다. (... ) ) ). 으흠흠흠 (?.


In [17]:
from transformers import BertTokenizer, AutoTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
        
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)

In [20]:
def train2(model, iterator, optimizer, criterion, scheduler,clip):
    
    model.train()
    
    epoch_loss = 0
    optimizer.zero_grad()
    #for i, batch in enumerate(iterator):
    for i in range(0,len(iterator)):
        batch = iterator[i]
        print(f'\r batch {i+1}/{len(iterator)}', end="", flush=True)

        input_ids = torch.tensor(batch["input_ids"]).to(device)
        attention_mask = torch.tensor(batch["attention_mask"]).to(device)
        decoder_input_ids= torch.tensor(batch["decoder_input_ids"]).to(device)
        decoder_attention_mask= torch.tensor(batch["decoder_attention_mask"]).to(device)
        labels= torch.tensor(batch["labels"]).to(device)

            
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                        decoder_input_ids=decoder_input_ids, 
                        decoder_attention_mask=decoder_attention_mask,
                        labels=labels)

        loss = outputs.loss #* b_rewards
        loss.backward()
        ##print('loss',loss)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        scheduler.step()
        
        epoch_loss += loss.item()
        ##print('epoch_loss',epoch_loss)

        if i%1000 == 0:
            print('\tTarget text:',generate_summary2('옛날 귀여운 여자 아기가 마음씨 소녀가 되었어요.'))      
            model.save_pretrained("/content/drive/MyDrive/GAN_ENDE/sentence_complete_model")
            model.train()
            
    return epoch_loss / len(iterator)

In [21]:
import time
import random
import datetime
import math

CLIP = 1

best_valid_loss = float('inf')
model.to(device)

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(dataset_iterator) * N_EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    train_loss = train2(model, dataset_iterator, optimizer, criterion, scheduler,CLIP)
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    #if valid_loss < best_valid_loss:
    #    best_valid_loss = valid_loss
    #    torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')

    doc_end_time = time.time()
    model.save_pretrained("/content/drive/MyDrive/GAN_ENDE/sentence_complete_model")

 batch 1/31250	Target text: 옛날 귀여운 여자 아기가 마음씨 좋은 소녀가 되었어요. ( ) 라고 말했습니다. (??? ) ). 으흠흠흠! ( (?? ) 으잉! )
 batch 1001/31250	Target text: 옛날 귀여운 여자 아기가 마음씨 좋은 소녀가 되어요..... ). (? ).
 batch 2001/31250	Target text: 옛날 귀여운 여자 아기가 마음씨 좋은 소녀가 되었어요. (??? ) 라고 합니다. (?. ). 으흠흠흠 으소소소 ( -! )
 batch 3001/31250	Target text: 옛날 귀여운 여자 아기가 마음씨 좋은 소녀가 되었어요.... ) (? ) ). (?? )! ( (??? )!!!!.
 batch 4001/31250	Target text: 옛날 귀여운 여자 아기가 마음씨 좋은 소녀가 되었고요.... ) (? ) ) (?? ).
 batch 5001/31250	Target text: 옛날 귀여운 여자 아기가 마음씨 좋은 소녀가 되어요! 라고 고백합니다.....
 batch 6001/31250	Target text: 옛날 귀여운 여자 아기가 마음씨 좋은 소녀가 되어요! 라고 말합니다. (? )!!!!??
 batch 7001/31250	Target text: 옛날 귀여운 여자 아기가 마음씨 좋은 소녀가 되어요! 라고 말했습니다. (? )! (?? )!!!!! (?? ) 으흠흠흠.
 batch 8001/31250	Target text: 옛날 귀여운 여자 아기가 마음씨 좋은 소녀가 되어요....
 batch 9001/31250	Target text: 옛날 귀여운 여자 아기가 마음씨 좋은 소녀가 되어요!...!!! (? ) (????? ) ( (?! )! 아
 batch 10001/31250	Target text: 옛날 귀여운 여자 아기가 마음씨 좋은 소녀가 되어요. 라고 말합니다.... (? ) 아. 라는 소녀가었어요. 고요.
 batch 11001/31250	Target text: 옛날 귀